# Models API (WIP)

## List of models supported by `timm`

`timm` supports a wide variety of pretrained and non-pretrained models for number of Image based tasks. 

To get a complete list of models, use the `list_models` function from `timm` as below. The `list_models` function returns a list of models ordered alphabetically that are supported by `timm`. We just look at the top-5 models below.

In [ ]:
import timm 

timm.list_models()[:5]

['adv_inception_v3',
 'cspdarknet53',
 'cspdarknet53_iabn',
 'cspresnet50',
 'cspresnet50d']

In general, you always want to use factory functions inside `timm`. Particularly, you want to use `create_model` function from `timm` to create any model. It is possible to create any of the models listed in `timm.list_models()` using the `create_model` function. There are also some wonderful extra features that we will look at later. But, let's see a quick example. 

In [ ]:
import random
import torch

random_model_to_create = random.choice(timm.list_models())
random_model_to_create

'tf_efficientnet_b4'

In [ ]:
model = timm.create_model(random_model_to_create)
x     = torch.randn(1, 3, 224, 224)
model(x).shape

torch.Size([1, 1000])

In the example above, we randomly select a model name in `timm.list_models()`, create it and pass some dummy input data through the model to get some output. In general, you never want to create random models like this, and it's only an example to showcase that all models in `timm.list_models()` are supported by `timm.create_model()` function. It's really that easy to create a model using `timm`. 

## Does `timm` have pretrained weights for these models?

Of course! `timm` wants to make it super easy for researchers and practioners to experiment and supports a whole lot of models with pretrained weights. These pretrained weights are either:

1. Directly used from their original sources
2. Ported by Ross from their original implementation in a different framework (e.g. Tensorflow models)
3. Trained from scratch using the included training script (`train.py`). The exact commands with hyperparameters to train these individual models are mentioned under `Training Scripts`.

To list all the models that have pretrained weights, `timm` provides a convenience parameter `pretrained` that could be passed in `list_models` function as below. We only list the top-5 returned models.

In [ ]:
timm.list_models(pretrained=True)[:5]

['adv_inception_v3',
 'cspdarknet53',
 'cspresnet50',
 'cspresnext50',
 'densenet121']

> NOTE: Just by listing the top-5 pretrained models, we can see that `timm` does not currently have pretrained weights for models such as `cspdarknet53_iabn` or `cspresnet50d`. This is a great opportunity for new contributors with hardware availability to pretrain the models on Imagenet dataset using the training script and share these weights. 

## My dataset doesn't consist of 3-channel images - what now?

As you might already know, ImageNet data consists of 3-chanenl RGB images. Therefore, to be able to use pretrained weights in most libraries, the model expects a 3-channel input image. 

### `torchvision` raises `Exception`

In [ ]:
import torchvision

m = torchvision.models.resnet34(pretrained=True)

# single-channel image (maybe x-ray)
x = torch.randn(1, 1, 224, 224)

# `torchvision` raises error
try: m(x).shape
except Exception as e: print(e)

Given groups=1, weight of size [64, 3, 7, 7], expected input[1, 1, 224, 224] to have 3 channels, but got 1 channels instead


As can be seen above, these pretrained weights from `torchvision` won't work with single channel input images. As a work around most practitioners convert their single channel input images to 3-channel images by copying the single channel pixels accross to create a 3-channel image. 

Basically, `torchvision` above is complaining that it expects the input to have 3 channels, but got 1 channel instead. 

In [ ]:

# 25-channel image (maybe satellite image)
x = torch.randn(1, 25, 224, 224)

# `torchvision` raises error
try: m(x).shape
except Exception as e: print(e)

Given groups=1, weight of size [64, 3, 7, 7], expected input[1, 25, 224, 224] to have 3 channels, but got 25 channels instead


Again, `torchvision` raises an error and this time there is no workaround to get past this error apart from just not using pretrained weights and starting with randomly initialized weights. 

### `timm` has a way to handle these `exceptions`

In [ ]:
m = timm.create_model('resnet34', pretrained=True, in_chans=1)

# single channel image
x = torch.randn(1, 1, 224, 224)

m(x).shape

torch.Size([1, 1000])

We pass in a parameter `in_chans` to the `timm.create_model` function and this somehow just magically works! Let's see what happens with the 25-channel image?

In [ ]:
m = timm.create_model('resnet34', pretrained=True, in_chans=25)

# 25-channel image
x = torch.randn(1, 25, 224, 224)

m(x).shape

torch.Size([1, 1000])

This works again! :) 

### How is `timm` able to use pretrained weights and handle images that are not 3-channel RGB images? 